In [160]:
import numpy as np
import pandas as pd

Due to the recent epidemic of West Nile Virus in the Windy City, we've had the Department of Public Health set up a surveillance and control system. We're hoping it will let us learn something from the mosquito population as we collect data over time. Pesticides are a necessary evil in the fight for public health and safety, not to mention expensive! We need to derive an effective plan to deploy pesticides throughout the city, and that is exactly where you come in!

In this competition, you will be **analyzing weather data and GIS data and predicting whether or not West Nile virus is present, for a given time, location, and species.** 

Every year from late-May to early-October, public health workers in Chicago setup mosquito traps scattered across the city. Every week from Monday through Wednesday, these traps collect mosquitos, and the mosquitos are tested for the presence of West Nile virus before the end of the week. The test results include the number of mosquitos, the mosquitos species, and whether or not West Nile virus is present in the cohort. 

**Modeling**

- The goal is of course to **build a model and make predictions that the city of Chicago can use when it decides where to spray pesticides!** Your team should have a clean Jupyter Notebook that shows your EDA process, your modeling and predictions.

- **Conduct a cost-benefit analysis. This should include annual cost projections for various levels of pesticide coverage (cost) and the effect of these various levels of pesticide coverage (benefit).** (Hint: How would we quantify the benefit of pesticide spraying? To get "maximum benefit," what does that look like and how much does that cost? What if we cover less and therefore get a lower level of benefit?)

*From Kaggle website:*

**train.csv, test.csv** - the training and test set of the main dataset.

The training set consists of data from 2007, 2009, 2011, and 2013, while in the test set you are requested to predict the test results for 2008, 2010, 2012, and 2014.

- Id: the id of the record
- Date: date that the WNV test is performed
- Address: approximate address of the location of trap. This is used to send to the GeoCoder. 
- Species: the species of mosquitos
- Block: block number of address
- Street: street name
- Trap: Id of the trap
- AddressNumberAndStreet: approximate address returned from GeoCoder
- Latitude, Longitude: Latitude and Longitude returned from GeoCoder
- AddressAccuracy: accuracy returned from GeoCoder
- NumMosquitos: number of mosquitoes caught in this trap
- WnvPresent: whether West Nile Virus was present in these mosquitos. 1 means WNV is present, and 0 means not present. 

**spray.csv** - GIS data of spraying efforts in 2011 and 2013

- Date, Time: the date and time of the spray
- Latitude, Longitude: the Latitude and Longitude of the spray

**weather.csv** - weather data from 2007 to 2014. Column descriptions in noaa_weather_qclcd_documentation.pdf. 

**sampleSubmission.csv** - a sample submission file in the correct format

#### Main dataset (Train & Test)

These test results are organized in such a way that when the number of mosquitos exceed 50, they are split into another record (another row in the dataset), such that the number of mosquitos are capped at 50. 

The location of the traps are described by the block number and street name. For your convenience, we have mapped these attributes into Longitude and Latitude in the dataset. Please note that these are derived locations. For example, Block=79, and Street= "W FOSTER AVE" gives us an approximate address of "7900 W FOSTER AVE, Chicago, IL", which translates to (41.974089,-87.824812) on the map.

Some traps are "satellite traps". These are traps that are set up near (usually within 6 blocks) an established trap to enhance surveillance efforts. Satellite traps are postfixed with letters. For example, T220A is a satellite trap to T220. 

Please note that not all the locations are tested at all times. Also, records exist only when a particular species of mosquitos is found at a certain trap at a certain time. In the test set, we ask you for all combinations/permutations of possible predictions and are only scoring the observed ones.

In [161]:
train = pd.read_csv('input/train.csv')
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [162]:
spray = pd.read_csv('input/spray.csv')
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [163]:
weather = pd.read_csv('input/weather.csv')
weather.head().T

,0,1,2,3,4
Station,1,2,1,2,1
Date,2007-05-01,2007-05-01,2007-05-02,2007-05-02,2007-05-03
Tmax,83,84,59,60,66
Tmin,50,52,42,43,46
Tavg,67,68,51,52,56
Depart,14,M,-3,M,2
DewPoint,51,51,42,42,40
WetBulb,56,57,47,47,48
Heat,0,0,14,13,9
Cool,2,3,0,0,0


# Clean/explore train dataset first

In [164]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [165]:
train.isnull().sum().sort_values(ascending=False)

# no nulls at all

WnvPresent                0
NumMosquitos              0
AddressAccuracy           0
Longitude                 0
Latitude                  0
AddressNumberAndStreet    0
Trap                      0
Street                    0
Block                     0
Species                   0
Address                   0
Date                      0
dtype: int64

In [166]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
Date                      10506 non-null object
Address                   10506 non-null object
Species                   10506 non-null object
Block                     10506 non-null int64
Street                    10506 non-null object
Trap                      10506 non-null object
AddressNumberAndStreet    10506 non-null object
Latitude                  10506 non-null float64
Longitude                 10506 non-null float64
AddressAccuracy           10506 non-null int64
NumMosquitos              10506 non-null int64
WnvPresent                10506 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 985.1+ KB


In [167]:
train.describe()

,Block,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
count,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000
mean,35.687797,41.841139,-87.699908,7.819532,12.853512,0.052446
std,24.339468,0.112742,0.096514,1.452921,16.133816,0.222936
min,10.000000,41.644612,-87.930995,3.000000,1.000000,0.000000
25%,12.000000,41.732984,-87.760070,8.000000,2.000000,0.000000
50%,33.000000,41.846283,-87.694991,8.000000,5.000000,0.000000
75%,52.000000,41.954690,-87.627796,9.000000,17.000000,0.000000
max,98.000000,42.017430,-87.531635,9.000000,50.000000,1.000000


In [168]:
train.WnvPresent.value_counts(normalize=True)

0    0.947554
1    0.052446
Name: WnvPresent, dtype: float64

Special mention on the target feature `WnvPresent`: 

- It seems that train dataset is highly imbalanced with only 5% positive detection of WNV virus.
    - Might need to balance out the dataset later using techniques such as bootstrapping, oversampling, SMOTE, etc.

In [169]:
train.dtypes

Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
dtype: object

In [170]:
for x in train.columns:
    print(f"{x}'s unique values are:", train[x].unique())
    print("\n\n")

Date's unique values are: ['2007-05-29' '2007-06-05' '2007-06-26' '2007-06-29' '2007-07-02'
 '2007-07-11' '2007-07-18' '2007-07-19' '2007-07-25' '2007-07-27'
 '2007-08-01' '2007-08-02' '2007-08-03' '2007-08-07' '2007-08-08'
 '2007-08-09' '2007-08-15' '2007-08-16' '2007-08-17' '2007-08-21'
 '2007-08-22' '2007-08-24' '2007-08-28' '2007-09-04' '2007-09-05'
 '2007-09-06' '2007-09-12' '2007-09-18' '2007-09-19' '2007-09-24'
 '2007-09-25' '2007-10-04' '2007-10-09' '2009-05-28' '2009-06-02'
 '2009-06-03' '2009-06-05' '2009-06-12' '2009-06-15' '2009-06-19'
 '2009-06-22' '2009-06-26' '2009-06-29' '2009-07-06' '2009-07-10'
 '2009-07-13' '2009-07-17' '2009-07-24' '2009-07-27' '2009-07-31'
 '2009-08-07' '2009-08-13' '2009-08-25' '2009-08-27' '2009-09-03'
 '2009-09-14' '2009-09-17' '2009-09-25' '2009-10-01' '2011-06-10'
 '2011-06-17' '2011-06-24' '2011-06-30' '2011-07-11' '2011-07-15'
 '2011-07-25' '2011-07-29' '2011-08-05' '2011-08-12' '2011-08-19'
 '2011-08-26' '2011-09-01' '2011-09-02' '2011-09-1

In [171]:
# get dummies for species first:
train = pd.get_dummies(train, columns=['Species'], drop_first=True)    

# convert 'Date' feature to actual datetime format:
train.Date = pd.to_datetime(train.Date)

train.head()

,Date,Address,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,0,1,0,0,0,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,0,0,1,0,0,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,0,0,1,0,0,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,0,1,0,0,0,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,0,0,1,0,0,0


# Clean/explore spray dataset (before merging with train)

In [172]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [173]:
spray.isnull().sum().sort_values(ascending=False)

Time         584
Longitude      0
Latitude       0
Date           0
dtype: int64

There are some nulls in `Time`, but since we are aiming to merge with train dataset on `Lattitude` and `Longitude`, we can leave the nulls as they are for the time being.

In [174]:
spray.dtypes

Date          object
Time          object
Latitude     float64
Longitude    float64
dtype: object

In [175]:
spray.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
Date         14835 non-null object
Time         14251 non-null object
Latitude     14835 non-null float64
Longitude    14835 non-null float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [176]:
spray.describe()

,Latitude,Longitude
count,14835.000000,14835.000000
mean,41.904828,-87.736690
std,0.104381,0.067292
min,41.713925,-88.096468
25%,41.785001,-87.794225
50%,41.940075,-87.727853
75%,41.980978,-87.694108
max,42.395983,-87.586727


In [177]:
for x in spray.columns:
    print(f"{x}'s unique values are:", spray[x].unique())
    print("\n\n")

Date's unique values are: ['2011-08-29' '2011-09-07' '2013-07-17' '2013-07-25' '2013-08-08'
 '2013-08-15' '2013-08-16' '2013-08-22' '2013-08-29' '2013-09-05']



Time's unique values are: ['6:56:58 PM' '6:57:08 PM' '6:57:18 PM' ... '8:04:01 PM' '8:04:11 PM'
 '8:04:21 PM']



Latitude's unique values are: [42.39162333 42.39134833 42.39102167 ... 42.00602167 42.00545333
 42.004805  ]



Longitude's unique values are: [-88.08916333 -88.08915667 -88.08915833 ... -87.81739167 -87.81742333
 -87.81746   ]





While we try to merge with train on `Lattitude` and `Longitude`, but it is hard to match both features on each dataframe exactly, due to the very high resolution and exactness of the geo-location values.

As such, round off the values in both dataframes to 3 decimal places for an easier match and merge.

In [178]:
spray['Latitude'] = spray['Latitude'].round(3)
spray['Longitude'] = spray['Longitude'].round(3)

train['Longitude'] = train['Longitude'].round(3)
train['Latitude'] = train['Latitude'].round(3)

In [179]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.392,-88.089
1,2011-08-29,6:57:08 PM,42.391,-88.089
2,2011-08-29,6:57:18 PM,42.391,-88.089
3,2011-08-29,6:57:28 PM,42.391,-88.089
4,2011-08-29,6:57:38 PM,42.390,-88.089


In [180]:
train.head()

,Date,Address,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.955,-87.801,9,1,0,0,1,0,0,0,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.955,-87.801,9,1,0,0,0,1,0,0,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.995,-87.769,9,1,0,0,0,1,0,0,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974,-87.825,8,1,0,0,1,0,0,0,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974,-87.825,8,4,0,0,0,1,0,0,0


In [181]:
spray['Date'] = pd.to_datetime(spray['Date']) # standardize with train DataFrame
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.392,-88.089
1,2011-08-29,6:57:08 PM,42.391,-88.089
2,2011-08-29,6:57:18 PM,42.391,-88.089
3,2011-08-29,6:57:28 PM,42.391,-88.089
4,2011-08-29,6:57:38 PM,42.390,-88.089


In [182]:
print(spray.shape)
print(train.shape)

(14835, 4)
(10506, 17)


In [183]:
train.columns

Index(['Date', 'Address', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy', 'NumMosquitos', 'WnvPresent', 'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS', 'Species_CULEX RESTUANS', 'Species_CULEX SALINARIUS', 'Species_CULEX TARSALIS', 'Species_CULEX TERRITANS'], dtype='object')

In [184]:
# calculate the count, max, min, peak-to-peak values for each long/lat geo-location.

spray = spray.groupby(['Longitude','Latitude']).Date.agg(['count','min','max']).reset_index()

# 'count' represents how many times a place was sprayed
# 'min' and 'max' represent the earliest and latest dates when the place was sprayed

spray['diff'] = spray['max'] - spray['min']
spray['diff'] = spray['diff'].dt.days
spray

# obtain the spread of the days for spraying each site

,Longitude,Latitude,count,min,max,diff
0,-88.096,42.390,4,2011-08-29,2011-08-29,0
1,-88.096,42.391,5,2011-08-29,2011-08-29,0
2,-88.096,42.392,2,2011-08-29,2011-08-29,0
3,-88.096,42.393,3,2011-08-29,2011-08-29,0
4,-88.096,42.394,3,2011-08-29,2011-08-29,0
...,...,...,...,...,...,...
6512,-87.587,41.724,3,2013-08-22,2013-08-22,0
6513,-87.587,41.725,2,2013-08-22,2013-08-22,0
6514,-87.587,41.726,2,2013-08-22,2013-08-22,0
6515,-87.587,41.727,1,2013-08-22,2013-08-22,0


In [185]:
# sort dataframe by the spread of days, in descending order

spray = spray.sort_values('diff', ascending=False)
spray.head()

,Longitude,Latitude,count,min,max,diff
696,-87.811,41.997,3,2011-09-07,2013-09-05,729
1804,-87.784,41.957,3,2013-08-15,2013-08-16,1
1858,-87.783,41.965,3,2013-08-15,2013-08-16,1
2789,-87.750,41.911,2,2013-08-15,2013-08-16,1
1801,-87.784,41.954,4,2013-08-15,2013-08-16,1


In [186]:
#merge train with spray:
train_spray = pd.merge(spray, train, on = ['Longitude','Latitude'], how='right',indicator=True)

print(train.shape)
print(train_spray.shape)

(10506, 17)
(10506, 22)


In [187]:
print(spray.columns)
print(train_spray.columns)

Index(['Longitude', 'Latitude', 'count', 'min', 'max', 'diff'], dtype='object')
Index(['Longitude', 'Latitude', 'count', 'min', 'max', 'diff', 'Date', 'Address', 'Block', 'Street', 'Trap', 'AddressNumberAndStreet', 'AddressAccuracy', 'NumMosquitos', 'WnvPresent', 'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS', 'Species_CULEX RESTUANS', 'Species_CULEX SALINARIUS', 'Species_CULEX TARSALIS', 'Species_CULEX TERRITANS', '_merge'], dtype='object')


In [188]:
# transform date again:

train_spray['Date'] = pd.to_datetime(train_spray['Date'])
train_spray.head()

,Longitude,Latitude,count,min,max,diff,Date,Address,Block,Street,Trap,AddressNumberAndStreet,AddressAccuracy,NumMosquitos,WnvPresent,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,_merge
0,-87.705,41.883,3.0,2013-08-22,2013-08-22,0.0,2007-07-02,"3300 West Randolph Street, Chicago, IL 60612, USA",33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",8,2,0,0,1,0,0,0,0,both
1,-87.705,41.883,3.0,2013-08-22,2013-08-22,0.0,2007-07-02,"3300 West Randolph Street, Chicago, IL 60612, USA",33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",8,1,0,0,0,1,0,0,0,both
2,-87.705,41.883,3.0,2013-08-22,2013-08-22,0.0,2007-07-18,"3300 West Randolph Street, Chicago, IL 60612, USA",33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",8,1,0,0,1,0,0,0,0,both
3,-87.705,41.883,3.0,2013-08-22,2013-08-22,0.0,2007-07-18,"3300 West Randolph Street, Chicago, IL 60612, USA",33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",8,1,0,0,0,1,0,0,0,both
4,-87.705,41.883,3.0,2013-08-22,2013-08-22,0.0,2007-07-18,"3300 West Randolph Street, Chicago, IL 60612, USA",33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",8,1,0,0,0,1,0,0,0,both


In [189]:
# get difference between date that the WNV test is performed, 'Date', and date when spray was performed

train_spray['diffmin'] = train_spray['Date'] - train_spray['min']
train_spray['diffmin'] = train_spray['diffmin'].dt.days

train_spray['diffmax'] = train_spray['Date'] - train_spray['max']
train_spray['diffmax'] = train_spray['diffmax'].dt.days

In [190]:
train_spray.head()

,Longitude,Latitude,count,min,max,diff,Date,Address,Block,Street,Trap,AddressNumberAndStreet,AddressAccuracy,NumMosquitos,WnvPresent,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,_merge,diffmin,diffmax
0,-87.705,41.883,3.0,2013-08-22,2013-08-22,0.0,2007-07-02,"3300 West Randolph Street, Chicago, IL 60612, USA",33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",8,2,0,0,1,0,0,0,0,both,-2243.0,-2243.0
1,-87.705,41.883,3.0,2013-08-22,2013-08-22,0.0,2007-07-02,"3300 West Randolph Street, Chicago, IL 60612, USA",33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",8,1,0,0,0,1,0,0,0,both,-2243.0,-2243.0
2,-87.705,41.883,3.0,2013-08-22,2013-08-22,0.0,2007-07-18,"3300 West Randolph Street, Chicago, IL 60612, USA",33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",8,1,0,0,1,0,0,0,0,both,-2227.0,-2227.0
3,-87.705,41.883,3.0,2013-08-22,2013-08-22,0.0,2007-07-18,"3300 West Randolph Street, Chicago, IL 60612, USA",33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",8,1,0,0,0,1,0,0,0,both,-2227.0,-2227.0
4,-87.705,41.883,3.0,2013-08-22,2013-08-22,0.0,2007-07-18,"3300 West Randolph Street, Chicago, IL 60612, USA",33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",8,1,0,0,0,1,0,0,0,both,-2227.0,-2227.0


Feature engineer a new column `most_recent_spray`, which is the most recent spray in terms of number of days ago.

In [191]:
train_spray['most_recent_spray'] = 3650

# set all values to 3650 days, or 10 years, ago first

In [192]:
train_spray.notnull().sum()

Longitude                         10506
Latitude                          10506
count                               963
min                                 963
max                                 963
diff                                963
Date                              10506
Address                           10506
Block                             10506
Street                            10506
Trap                              10506
AddressNumberAndStreet            10506
AddressAccuracy                   10506
NumMosquitos                      10506
WnvPresent                        10506
Species_CULEX PIPIENS             10506
Species_CULEX PIPIENS/RESTUANS    10506
Species_CULEX RESTUANS            10506
Species_CULEX SALINARIUS          10506
Species_CULEX TARSALIS            10506
Species_CULEX TERRITANS           10506
_merge                            10506
diffmin                             963
diffmax                             963
most_recent_spray                 10506


In [193]:
train_spray.isnull().sum()

Longitude                            0
Latitude                             0
count                             9543
min                               9543
max                               9543
diff                              9543
Date                                 0
Address                              0
Block                                0
Street                               0
Trap                                 0
AddressNumberAndStreet               0
AddressAccuracy                      0
NumMosquitos                         0
WnvPresent                           0
Species_CULEX PIPIENS                0
Species_CULEX PIPIENS/RESTUANS       0
Species_CULEX RESTUANS               0
Species_CULEX SALINARIUS             0
Species_CULEX TARSALIS               0
Species_CULEX TERRITANS              0
_merge                               0
diffmin                           9543
diffmax                           9543
most_recent_spray                    0
dtype: int64

In [194]:
# drop irrelevant '_merge' column:

train_spray.drop(columns = '_merge', inplace=True)
train_spray.shape[1]

24

In order to create a feature with the most recent spray in days (and populate the entries with no diffmin/diffmax), there are now 963 non-null observations with spray, the rest is without data on spray.

Hence, perform an inner merge to focus on the spray/train intersection, thus avoiding dealing with the nulls.

Subsequently, concatenate the inner-merged dataframe with the above right-merged dataframe, by joining at the point where the null entries start appearing in the right-merged dataframe

In [195]:
# rename previous train_spray dataframe to prevent confusion:
train_spray_right = train_spray

# create inner merge dataframe
train_spray_inner = pd.merge(train, spray, on = ['Longitude','Latitude'], how='inner',indicator=True)

# perform the same transformations as above
train_spray_inner['Date'] = pd.to_datetime(train_spray_inner['Date'])
train_spray_inner['diffmin'] = train_spray_inner['Date'] - train_spray_inner['min']
train_spray_inner['diffmin'] = train_spray_inner['diffmin'].dt.days
train_spray_inner['diffmax'] = train_spray_inner['Date'] - train_spray_inner['max']
train_spray_inner['diffmax'] = train_spray_inner['diffmax'].dt.days

train_spray_inner.head()

,Date,Address,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,count,min,max,diff,_merge,diffmin,diffmax
0,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974,-87.825,8,1,0,0,1,0,0,0,0,1,2011-09-07,2011-09-07,0,both,-1562,-1562
1,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974,-87.825,8,4,0,0,0,1,0,0,0,1,2011-09-07,2011-09-07,0,both,-1562,-1562
2,2007-06-05,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974,-87.825,8,1,0,0,1,0,0,0,0,1,2011-09-07,2011-09-07,0,both,-1555,-1555
3,2007-06-05,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974,-87.825,8,2,0,0,0,1,0,0,0,1,2011-09-07,2011-09-07,0,both,-1555,-1555
4,2007-06-26,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974,-87.825,8,1,0,0,0,1,0,0,0,1,2011-09-07,2011-09-07,0,both,-1534,-1534


In [196]:
train_spray_inner.shape

(963, 24)

In [197]:
# Create a function that calculates the days that have passed since the most recent spray, 
# given a date of collection minus dates of spray:

def recent_spray(diffmin, diffmax):
    if (diffmin >= 0) & (diffmax < 0):
        days = diffmin
    elif (diffmax >= 0) & (diffmin < 0):
        days = diffmax
    elif (diffmax < 0) & (diffmin < 0):
        # rows with diffmax/diffmin means the spray only occurred after the data collection date
        days = 3650
    elif (diffmax >= 0) & (diffmin >= 0):
        if diffmax < diffmin:
            days = diffmax
        elif diffmax > diffmin:
            days = diffmin
        else:
            days = diffmin
    elif diffmin.isnull() or diffmax.isnull():
        # populate diffmin/diffmax nulls with 3650 (or 10 years ago)
        days = 3650
    return(days)

In [198]:
# feature engineer same 'most_recent_spray' column for train_spray_inner

train_spray_inner['most_recent_spray'] = train_spray_inner.apply(lambda x: recent_spray(x['diffmin'], x['diffmax']),axis=1)


In [199]:
train_spray_right[train_spray_right.isna().any(axis=1)]

# null entries start appearing from index 963 onwards

,Longitude,Latitude,count,min,max,diff,Date,Address,Block,Street,Trap,AddressNumberAndStreet,AddressAccuracy,NumMosquitos,WnvPresent,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,diffmin,diffmax,most_recent_spray
963,-87.801,41.955,NaN,NaT,NaT,NaN,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",9,1,0,0,1,0,0,0,0,NaN,NaN,3650
964,-87.801,41.955,NaN,NaT,NaT,NaN,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",9,1,0,0,0,1,0,0,0,NaN,NaN,3650
965,-87.801,41.955,NaN,NaT,NaT,NaN,2007-06-05,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",9,3,0,0,1,0,0,0,0,NaN,NaN,3650
966,-87.801,41.955,NaN,NaT,NaT,NaN,2007-06-05,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",9,5,0,0,0,1,0,0,0,NaN,NaN,3650
967,-87.801,41.955,NaN,NaT,NaT,NaN,2007-06-05,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",9,1,0,1,0,0,0,0,0,NaN,NaN,3650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10501,-87.634,41.926,NaN,NaT,NaT,NaN,2013-08-22,"2100 North Cannon Drive, Chicago, IL 60614, USA",21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",8,19,0,1,0,0,0,0,0,NaN,NaN,3650
10502,-87.634,41.926,NaN,NaT,NaT,NaN,2013-08-29,"2100 North Cannon Drive, Chicago, IL 60614, USA",21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",8,1,0,0,1,0,0,0,0,NaN,NaN,3650
10503,-87.634,41.926,NaN,NaT,NaT,NaN,2013-09-06,"2100 North Cannon Drive, Chicago, IL 60614, USA",21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",8,3,0,0,1,0,0,0,0,NaN,NaN,3650
10504,-87.634,41.926,NaN,NaT,NaT,NaN,2013-09-12,"2100 North Cannon Drive, Chicago, IL 60614, USA",21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",8,2,0,1,0,0,0,0,0,NaN,NaN,3650


In [200]:
# finally concat the two DataFrames:

train_spray = pd.concat([train_spray_inner, train_spray_right.loc[963:,:]])

/Users/ganeshsivam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [201]:
train_spray.columns

Index(['Address', 'AddressAccuracy', 'AddressNumberAndStreet', 'Block', 'Date', 'Latitude', 'Longitude', 'NumMosquitos', 'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS', 'Species_CULEX RESTUANS', 'Species_CULEX SALINARIUS', 'Species_CULEX TARSALIS', 'Species_CULEX TERRITANS', 'Street', 'Trap', 'WnvPresent', '_merge', 'count', 'diff', 'diffmax', 'diffmin', 'max', 'min', 'most_recent_spray'], dtype='object')

Recap: **train dataset features**

- Id: the id of the record
- Date: date that the WNV test is performed
- Address: approximate address of the location of trap. This is used to send to the GeoCoder.
- Species: the species of mosquitos **(dummified)**
- Block: block number of address
- Street: street name
- Trap: Id of the trap
- AddressNumberAndStreet: approximate address returned from GeoCoder
- Latitude, Longitude: Latitude and Longitude returned from GeoCoder
- AddressAccuracy: accuracy returned from GeoCoder
- NumMosquitos: number of mosquitoes caught in this trap
- WnvPresent: whether West Nile Virus was present in these mosquitos. 1 means WNV is present, and 0 means not present. **(target)**

In [202]:
train_spray.head().T

,0,1,2,3,4
Address,"7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA"
AddressAccuracy,8,8,8,8,8
AddressNumberAndStreet,"7900 W FOSTER AVE, Chicago, IL","7900 W FOSTER AVE, Chicago, IL","7900 W FOSTER AVE, Chicago, IL","7900 W FOSTER AVE, Chicago, IL","7900 W FOSTER AVE, Chicago, IL"
Block,79,79,79,79,79
Date,2007-05-29 00:00:00,2007-05-29 00:00:00,2007-06-05 00:00:00,2007-06-05 00:00:00,2007-06-26 00:00:00
Latitude,41.974,41.974,41.974,41.974,41.974
Longitude,-87.825,-87.825,-87.825,-87.825,-87.825
NumMosquitos,1,4,1,2,1
Species_CULEX PIPIENS,0,0,0,0,0
Species_CULEX PIPIENS/RESTUANS,1,0,1,0,0


Drop `AddrressAccuracy` and other address-duplicated features, keeping only `Address` in.

Also dropping `_merge` and `Trap` - the latter is merely the ID of the trap.

Lastly, since `most_recent_spray` has been feature engineered, the other features that it was derived from can be dropped as well - namely `min`, `max`, `diff`, `diffmin`, `diffmax`, `count`

In [203]:
# drop irrelevant columns:

train_spray.drop(columns = ['AddressAccuracy', 'AddressNumberAndStreet', 'Block', 'Street', '_merge', 'Trap',
                           'min', 'max', 'diff', 'diffmin', 'diffmax', 'count'], inplace=True)

# rearrange columns:

train_spray = train_spray[[x for x in train_spray.columns if x != 'WnvPresent'] + ['WnvPresent']]
train_spray.head().T

,0,1,2,3,4
Address,"7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA"
Date,2007-05-29 00:00:00,2007-05-29 00:00:00,2007-06-05 00:00:00,2007-06-05 00:00:00,2007-06-26 00:00:00
Latitude,41.974,41.974,41.974,41.974,41.974
Longitude,-87.825,-87.825,-87.825,-87.825,-87.825
NumMosquitos,1,4,1,2,1
Species_CULEX PIPIENS,0,0,0,0,0
Species_CULEX PIPIENS/RESTUANS,1,0,1,0,0
Species_CULEX RESTUANS,0,1,0,1,1
Species_CULEX SALINARIUS,0,0,0,0,0
Species_CULEX TARSALIS,0,0,0,0,0


In [204]:
train_spray.shape

(10506, 13)

In [205]:
# save DataFrame first:

train_spray.to_csv('output/train_spray.csv')

# Clean/explore weather dataset

In [206]:
weather.head().T

,0,1,2,3,4
Station,1,2,1,2,1
Date,2007-05-01,2007-05-01,2007-05-02,2007-05-02,2007-05-03
Tmax,83,84,59,60,66
Tmin,50,52,42,43,46
Tavg,67,68,51,52,56
Depart,14,M,-3,M,2
DewPoint,51,51,42,42,40
WetBulb,56,57,47,47,48
Heat,0,0,14,13,9
Cool,2,3,0,0,0


#### Weather Data

It is believed that hot and dry conditions are more favorable for West Nile virus than cold and wet. We provide you with the dataset from NOAA of the weather conditions of 2007 to 2014, during the months of the tests. 

- Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
- Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

In [207]:
weather.shape

(2944, 22)

In [208]:
weather.isnull().sum()

# no nulls

Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
Depth          0
Water1         0
SnowFall       0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

In [209]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
DewPoint       2944 non-null int64
WetBulb        2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
Depth          2944 non-null object
Water1         2944 non-null object
SnowFall       2944 non-null object
PrecipTotal    2944 non-null object
StnPressure    2944 non-null object
SeaLevel       2944 non-null object
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null object
dtypes: float64(1), int64(5), object(16)
memory usage: 506.1+ KB


In [210]:
weather.describe()

,Station,Tmax,Tmin,DewPoint,ResultSpeed,ResultDir
count,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000
mean,1.500000,76.166101,57.810462,53.457880,6.960666,17.494905
std,0.500085,11.461970,10.381939,10.675181,3.587527,10.063609
min,1.000000,41.000000,29.000000,22.000000,0.100000,1.000000
25%,1.000000,69.000000,50.000000,46.000000,4.300000,7.000000
50%,1.500000,78.000000,59.000000,54.000000,6.400000,19.000000
75%,2.000000,85.000000,66.000000,62.000000,9.200000,25.000000
max,2.000000,104.000000,83.000000,75.000000,24.100000,36.000000


In [211]:
for x in weather.columns:
    print(f"{x}'s unique values are:", weather[x].unique())
    print("\n\n")

Station's unique values are: [1 2]



Date's unique values are: ['2007-05-01' '2007-05-02' '2007-05-03' ... '2014-10-29' '2014-10-30'
 '2014-10-31']



Tmax's unique values are: [ 83  84  59  60  66  67  78  68  82  80  77  76  70  73  64  65  69  90
  62  61  71  79  87  89  88  75  85  86  81  72  63  91  92  93  74  94
  54  53  56  57  58  55  50  95  52  47  45  51  48  44  49  46  96  99
 100 101  97  98 102 103 104  42  41]



Tmin's unique values are: [50 52 42 43 46 48 49 51 53 54 47 60 61 63 56 59 44 57 45 55 66 65 70 68
 62 67 64 58 71 69 73 75 72 74 39 41 40 37 34 38 35 36 33 31 32 76 77 29
 78 79 80 81 82 83]



Tavg's unique values are: ['67' '68' '51' '52' '56' '58' 'M' '60' '59' '65' '70' '69' '71' '61' '55'
 '57' '73' '72' '53' '62' '63' '74' '75' '78' '76' '77' '66' '80' '64'
 '81' '82' '79' '85' '84' '83' '50' '49' '46' '48' '45' '54' '47' '44'
 '40' '41' '38' '39' '42' '37' '43' '86' '87' '89' '92' '88' '91' '93'
 '94' '90' '36']



Depart's unique values are: ['14'

Aim to merge weather dataset with train_spray on `Date`.

In [212]:
weather['Date'] = pd.to_datetime(weather['Date'])

But it is also possible to join datasets by feature engineering the weather station.

Firstly, need to identify which weather station to allocate for each different entry in the dataset. This is done by calculating the distance between the geo-location values (lattitude & longitude) and the given geo-locations of the weather stations:

- Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
- Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

In [213]:
train_spray.head().T

,0,1,2,3,4
Address,"7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA"
Date,2007-05-29 00:00:00,2007-05-29 00:00:00,2007-06-05 00:00:00,2007-06-05 00:00:00,2007-06-26 00:00:00
Latitude,41.974,41.974,41.974,41.974,41.974
Longitude,-87.825,-87.825,-87.825,-87.825,-87.825
NumMosquitos,1,4,1,2,1
Species_CULEX PIPIENS,0,0,0,0,0
Species_CULEX PIPIENS/RESTUANS,1,0,1,0,0
Species_CULEX RESTUANS,0,1,0,1,1
Species_CULEX SALINARIUS,0,0,0,0,0
Species_CULEX TARSALIS,0,0,0,0,0


In [214]:
stat1 = {'lat': 41.995, 'long': -87.933}
stat2 = {'lat': 41.786, 'long': -87.752}

In [215]:
def calc_dist(lat, long):
    return np.sqrt((lat)**2 + (long)**2)

In [216]:
def nearest_stat(lat,long):
    dist1 = calc_dist(lat - stat1['lat'], long - stat1['long'])
    dist2 = calc_dist(lat - stat2['lat'], long - stat2['long'])
    if dist1 < dist2:
        stat = 1
    else:
        stat = 2
    return stat

In [217]:
# feature engineer 'station' column:

train_spray['Station'] = train_spray.apply(lambda x: nearest_stat(x['Latitude'], x['Longitude']), axis=1)
train_spray.head().T

,0,1,2,3,4
Address,"7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA"
Date,2007-05-29 00:00:00,2007-05-29 00:00:00,2007-06-05 00:00:00,2007-06-05 00:00:00,2007-06-26 00:00:00
Latitude,41.974,41.974,41.974,41.974,41.974
Longitude,-87.825,-87.825,-87.825,-87.825,-87.825
NumMosquitos,1,4,1,2,1
Species_CULEX PIPIENS,0,0,0,0,0
Species_CULEX PIPIENS/RESTUANS,1,0,1,0,0
Species_CULEX RESTUANS,0,1,0,1,1
Species_CULEX SALINARIUS,0,0,0,0,0
Species_CULEX TARSALIS,0,0,0,0,0


In [218]:
train_spray['Station'].value_counts(normalize=True)

2    0.782696
1    0.217304
Name: Station, dtype: float64

78.2% of entries are closer to weather station 2.

In [219]:
# finally, merge weather dataframe onto train_spray:

train_spray_weat = pd.merge(train_spray, weather, left_on=['Date','Station'], right_on=['Date','Station'], how='left')

In [220]:
train_spray_weat.head().T

,0,1,2,3,4
Address,"7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA"
Date,2007-05-29 00:00:00,2007-05-29 00:00:00,2007-06-05 00:00:00,2007-06-05 00:00:00,2007-06-26 00:00:00
Latitude,41.974,41.974,41.974,41.974,41.974
Longitude,-87.825,-87.825,-87.825,-87.825,-87.825
NumMosquitos,1,4,1,2,1
Species_CULEX PIPIENS,0,0,0,0,0
Species_CULEX PIPIENS/RESTUANS,1,0,1,0,0
Species_CULEX RESTUANS,0,1,0,1,1
Species_CULEX SALINARIUS,0,0,0,0,0
Species_CULEX TARSALIS,0,0,0,0,0


In [221]:
train_spray_weat.shape

(10506, 34)

In [222]:
train_spray_weat.columns

Index(['Address', 'Date', 'Latitude', 'Longitude', 'NumMosquitos', 'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS', 'Species_CULEX RESTUANS', 'Species_CULEX SALINARIUS', 'Species_CULEX TARSALIS', 'Species_CULEX TERRITANS', 'most_recent_spray', 'WnvPresent', 'Station', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth', 'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed'], dtype='object')

In [223]:

train_spray_weat.shape

(10506, 34)

From the [documentation](https://github.com/sbussmann/west-nile-virus/blob/master/noaa_weather_qclcd_documentation.pdf), certain weather features have 'M' or '-' as missing values.

Hence, explore and identify these missing values:

In [224]:
train_spray_weat.isin(['-']).sum()

Address                              0
Date                                 0
Latitude                             0
Longitude                            0
NumMosquitos                         0
Species_CULEX PIPIENS                0
Species_CULEX PIPIENS/RESTUANS       0
Species_CULEX RESTUANS               0
Species_CULEX SALINARIUS             0
Species_CULEX TARSALIS               0
Species_CULEX TERRITANS              0
most_recent_spray                    0
WnvPresent                           0
Station                              0
Tmax                                 0
Tmin                                 0
Tavg                                 0
Depart                               0
DewPoint                             0
WetBulb                              0
Heat                                 0
Cool                                 0
Sunrise                           8223
Sunset                            8223
CodeSum                              0
Depth                    

In [225]:
train_spray_weat.isin(['M']).sum()

Address                               0
Date                                  0
Latitude                              0
Longitude                             0
NumMosquitos                          0
Species_CULEX PIPIENS                 0
Species_CULEX PIPIENS/RESTUANS        0
Species_CULEX RESTUANS                0
Species_CULEX SALINARIUS              0
Species_CULEX TARSALIS                0
Species_CULEX TERRITANS               0
most_recent_spray                     0
WnvPresent                            0
Station                               0
Tmax                                  0
Tmin                                  0
Tavg                                  0
Depart                             8223
DewPoint                              0
WetBulb                              26
Heat                                  0
Cool                                  0
Sunrise                               0
Sunset                                0
CodeSum                               0


`Sunrise`, `Sunset`, `Depth`, `SnowFall`, `Depart` and `Water1` all are mostly made up for missing/null values. Hence, drop these columns.

In [226]:
train_spray_weat.drop(['Sunrise', 'Sunset', 'Water1', 'Depart', 'Depth','SnowFall'], axis=1, inplace=True)
train_spray_weat.shape

(10506, 28)

Interestingly, the same number of 566 null entries appear for `Depart`, `Sunrise` and `Sunset`.

In [227]:
train_spray_weat.isin(['-']).sum()

Address                           0
Date                              0
Latitude                          0
Longitude                         0
NumMosquitos                      0
Species_CULEX PIPIENS             0
Species_CULEX PIPIENS/RESTUANS    0
Species_CULEX RESTUANS            0
Species_CULEX SALINARIUS          0
Species_CULEX TARSALIS            0
Species_CULEX TERRITANS           0
most_recent_spray                 0
WnvPresent                        0
Station                           0
Tmax                              0
Tmin                              0
Tavg                              0
DewPoint                          0
WetBulb                           0
Heat                              0
Cool                              0
CodeSum                           0
PrecipTotal                       0
StnPressure                       0
SeaLevel                          0
ResultSpeed                       0
ResultDir                         0
AvgSpeed                    

In [228]:
train_spray_weat.isin(['M']).sum()

Address                            0
Date                               0
Latitude                           0
Longitude                          0
NumMosquitos                       0
Species_CULEX PIPIENS              0
Species_CULEX PIPIENS/RESTUANS     0
Species_CULEX RESTUANS             0
Species_CULEX SALINARIUS           0
Species_CULEX TARSALIS             0
Species_CULEX TERRITANS            0
most_recent_spray                  0
WnvPresent                         0
Station                            0
Tmax                               0
Tmin                               0
Tavg                               0
DewPoint                           0
WetBulb                           26
Heat                               0
Cool                               0
CodeSum                            0
PrecipTotal                       35
StnPressure                       26
SeaLevel                           0
ResultSpeed                        0
ResultDir                          0
A

There still remain some nulls, but not a very sizeable portion. Will attempt to impute these missing values accordingly, namely:

- `WetBulb`: 26 nulls
- `StnPressure`: 26 nulls
- `PrecipTotal`: 35 nulls

In [229]:
print(train_spray_weat.PrecipTotal.unique())
print(train_spray_weat.WetBulb.unique())
print(train_spray_weat.StnPressure.unique())

['0.00' '0.42' '0.16' '1.55' '0.17' '1.31' '0.23' '  T' '0.24' '0.27'
 '0.92' '0.06' '0.83' '0.01' '0.20' 'M' '0.87' '0.12' '0.02' '0.88' '0.03'
 '1.19' '0.36' '0.11' '0.84' '3.97' '0.08' '0.14' '0.33' '0.95' '0.19'
 '0.13' '0.59' '0.04' '0.44' '0.89' '0.52' '0.39' '0.70' '0.09' '0.79'
 '0.50' '0.58' '0.46']
['65' '51' '72' '59' '58' '71' '70' '69' '76' '66' '49' '68' '61' '52'
 '62' '60' '67' '50' '54' '55' '57' '63' '73' '64' '74' '75' 'M' '56' '47'
 '53' '46']
['29.39' '29.10' '29.41' '29.49' '29.23' '29.11' '29.29' '29.06' '29.17'
 '29.21' '29.28' '29.40' '29.34' '29.27' '29.44' '29.16' '29.45' '29.18'
 '29.26' '29.47' '29.46' '29.42' '29.30' '29.03' '29.20' '29.48' '29.51'
 '29.13' '29.32' '29.24' '29.38' '28.97' '29.31' '29.33' '29.36' 'M'
 '29.14' '29.15' '29.08' '29.25' '29.65' '29.55' '29.37' '29.12' '29.22'
 '29.59' '29.01' '29.19' '29.35' '28.95' '29.09' '28.89' '28.91']


According to [online research](http://theweatherprediction.com/habyhints/170/), it is possible to approximate missing values of `WetBulb` from `DewPoint` and temperature with this formula:

- Tavg - ((Tavg-DewPoint)/3)

In [230]:
train_spray_weat.WetBulb.unique()

array(['65', '51', '72', '59', '58', '71', '70', '69', '76', '66', '49',
       '68', '61', '52', '62', '60', '67', '50', '54', '55', '57', '63',
       '73', '64', '74', '75', 'M', '56', '47', '53', '46'], dtype=object)

In [231]:
def fill_wet_bulb(tavg, dp, wetbulb):
    if wetbulb == 'M':
        wetbulb = tavg - ((tavg-dp)/3)
    return wetbulb

In [232]:
# convert to correct datatype first:

train_spray_weat.dtypes

Address                                   object
Date                              datetime64[ns]
Latitude                                 float64
Longitude                                float64
NumMosquitos                               int64
Species_CULEX PIPIENS                      uint8
Species_CULEX PIPIENS/RESTUANS             uint8
Species_CULEX RESTUANS                     uint8
Species_CULEX SALINARIUS                   uint8
Species_CULEX TARSALIS                     uint8
Species_CULEX TERRITANS                    uint8
most_recent_spray                          int64
WnvPresent                                 int64
Station                                    int64
Tmax                                       int64
Tmin                                       int64
Tavg                                      object
DewPoint                                   int64
WetBulb                                   object
Heat                                      object
Cool                

In [233]:
train_spray_weat['Tavg'] = train_spray_weat['Tavg'].astype(int)
train_spray_weat['Heat'] = train_spray_weat['Heat'].astype(int)
train_spray_weat['Cool'] = train_spray_weat['Cool'].astype(int)
train_spray_weat['SeaLevel'] = train_spray_weat['SeaLevel'].astype(float)
train_spray_weat['AvgSpeed'] = train_spray_weat['AvgSpeed'].astype(float)

In [234]:
# fill in missing WetBulb values by applying function to column:

train_spray_weat['WetBulb'] = train_spray_weat.apply(lambda x: fill_wet_bulb(x['Tavg'], x['DewPoint'], x['WetBulb']), axis=1)

train_spray_weat.WetBulb.unique()

array(['65', '51', '72', '59', '58', '71', '70', '69', '76', '66', '49',
       '68', '61', '52', '62', '60', '67', '50', '54', '55', '57', '63',
       '73', '64', '74', '75', 72.0, '56', '47', '53', '46'], dtype=object)

In [235]:
train_spray_weat['WetBulb'] = train_spray_weat['WetBulb'].astype(int)

For the remaining nulls in `PrecipTotal` and `StnPressure`, since number of missing values are small, mode or median would make sense as replacement.

In [236]:
import statistics as st
mode = st.mode(train_spray_weat['PrecipTotal'])
mode

'0.00'

In [237]:
train_spray_weat['PrecipTotal'] = train_spray_weat['PrecipTotal'].apply(lambda x: mode if x=='M' else x)

In [238]:
train_spray_weat.PrecipTotal.unique()

array(['0.00', '0.42', '0.16', '1.55', '0.17', '1.31', '0.23', '  T',
       '0.24', '0.27', '0.92', '0.06', '0.83', '0.01', '0.20', '0.87',
       '0.12', '0.02', '0.88', '0.03', '1.19', '0.36', '0.11', '0.84',
       '3.97', '0.08', '0.14', '0.33', '0.95', '0.19', '0.13', '0.59',
       '0.04', '0.44', '0.89', '0.52', '0.39', '0.70', '0.09', '0.79',
       '0.50', '0.58', '0.46'], dtype=object)

In [239]:
sorted(train_spray_weat.PrecipTotal.unique())

['  T',
 '0.00',
 '0.01',
 '0.02',
 '0.03',
 '0.04',
 '0.06',
 '0.08',
 '0.09',
 '0.11',
 '0.12',
 '0.13',
 '0.14',
 '0.16',
 '0.17',
 '0.19',
 '0.20',
 '0.23',
 '0.24',
 '0.27',
 '0.33',
 '0.36',
 '0.39',
 '0.42',
 '0.44',
 '0.46',
 '0.50',
 '0.52',
 '0.58',
 '0.59',
 '0.70',
 '0.79',
 '0.83',
 '0.84',
 '0.87',
 '0.88',
 '0.89',
 '0.92',
 '0.95',
 '1.19',
 '1.31',
 '1.55',
 '3.97']

Once again, according to [documentation](https://github.com/sbussmann/west-nile-virus/blob/master/noaa_weather_qclcd_documentation.pdf), 'T' stands for Trace amount.

For simplicity's sake, impute T to be between 0.000 and 0.004 (i.e. value of 0.002).

Logically, we assume if `PrecipTotal` to be absolutely 0, the value would have been 0.00 (and not T). If `PrecipTotal` was 0.005 and above, we assume it would be rounded up to 0.01. Hence, for T to be imputed, the value should be between 0.000 and 0.004, in which we will simply impute as the mean value **0.002**.

In [240]:
train_spray_weat['PrecipTotal'] = train_spray_weat['PrecipTotal'].apply(lambda x: 0.002 if x=='  T' else x)
train_spray_weat.PrecipTotal.unique()

array(['0.00', '0.42', '0.16', '1.55', '0.17', '1.31', '0.23', 0.002,
       '0.24', '0.27', '0.92', '0.06', '0.83', '0.01', '0.20', '0.87',
       '0.12', '0.02', '0.88', '0.03', '1.19', '0.36', '0.11', '0.84',
       '3.97', '0.08', '0.14', '0.33', '0.95', '0.19', '0.13', '0.59',
       '0.04', '0.44', '0.89', '0.52', '0.39', '0.70', '0.09', '0.79',
       '0.50', '0.58', '0.46'], dtype=object)

In [241]:
# finally convert feature to float:

train_spray_weat['PrecipTotal'] = train_spray_weat['PrecipTotal'].astype(float)

In [242]:
# impute M to be mode as well, for 'StnPressure', before converting to float:

mode = st.mode(train_spray_weat['StnPressure'])
train_spray_weat['StnPressure'] = train_spray_weat['StnPressure'].apply(lambda x: mode if x=='M' else x)

train_spray_weat['StnPressure'] = train_spray_weat['StnPressure'].astype(float)

Feature engineering: split `Date` into `Month` and `Year` to spot for seasonality trends.

Intuitively, months in the middle of the year would be hotter and hence, more conducive for mosquito breeding, resulting in higher frequency of WNV.

Also, with the global warming trends, mosquitoes trapped and WNV incidence may also be on the rise as the years go by.

In [243]:
train_spray_weat['Month'] = train_spray_weat['Date'].apply(lambda x: x.to_pydatetime().month)
train_spray_weat['Year'] = train_spray_weat['Date'].apply(lambda x: x.to_pydatetime().year)

train_spray_weat.head()

,Address,Date,Latitude,Longitude,NumMosquitos,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,most_recent_spray,WnvPresent,Station,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Month,Year
0,"7900 West Foster Avenue, Chicago, IL 60656, USA",2007-05-29,41.974,-87.825,1,0,1,0,0,0,0,3650,0,1,88,60,74,58,65,0,9,BR HZ,0.00,29.39,30.11,5.8,18,6.5,5,2007
1,"7900 West Foster Avenue, Chicago, IL 60656, USA",2007-05-29,41.974,-87.825,4,0,0,1,0,0,0,3650,0,1,88,60,74,58,65,0,9,BR HZ,0.00,29.39,30.11,5.8,18,6.5,5,2007
2,"7900 West Foster Avenue, Chicago, IL 60656, USA",2007-06-05,41.974,-87.825,1,0,1,0,0,0,0,3650,0,1,64,47,56,48,51,9,0,RA BR,0.42,29.10,29.79,5.2,5,7.6,6,2007
3,"7900 West Foster Avenue, Chicago, IL 60656, USA",2007-06-05,41.974,-87.825,2,0,0,1,0,0,0,3650,0,1,64,47,56,48,51,9,0,RA BR,0.42,29.10,29.79,5.2,5,7.6,6,2007
4,"7900 West Foster Avenue, Chicago, IL 60656, USA",2007-06-26,41.974,-87.825,1,0,0,1,0,0,0,3650,0,1,92,70,81,69,72,0,16,TSRA BR HZ VCTS,0.16,29.39,30.11,6.1,22,7.1,6,2007


We will also convert the weather types documented in `CodeSum` to account for the role that weather may play in WNV incidence, since weather will inevitably affect the environment suitability for mosquito breeding (hot and dry preferred to cold and wet).

In [244]:
train_spray_weat.CodeSum.unique()

array(['BR HZ', 'RA BR', 'TSRA BR HZ VCTS', ' ', 'TSRA RA BR VCTS',
       'RA BR HZ', 'HZ', 'TSRA RA BR', 'TSRA RA', 'TS TSRA RA BR HZ VCTS',
       'VCTS', 'TS TSRA RA BR', 'BR', 'RA', 'RA DZ BR', 'TS TSRA RA VCTS',
       'TSRA BR', 'TSRA', 'TS', 'FG BR HZ', 'TS TSRA BR', 'TS TSRA BR HZ',
       'DZ BR', 'TS BR', 'TS TSRA RA FG BR HZ', 'TSRA BR HZ',
       'TSRA RA DZ BR HZ', 'TSRA HZ'], dtype=object)

As per the [documentation](https://github.com/sbussmann/west-nile-virus/blob/master/noaa_weather_qclcd_documentation.pdf) once more, these are what the alphabets represent:

- HZ (Haze)
- FU (Smoke)
- VA (Volcanic Ash)
- +FC (Tornado)
- FC (Funnel Cloud)
- TS (Thunderstorm)
- GR (Hail)
- RA (Rain)
- DZ (Drizzle)
- SN (Snow)
- SG (Snow Grains)
- GS (Small Hail)
- PL (Ice Pellets)
- IC (Ice Crystals)
- FG (Fog), BR (Mist)
- UP (Unknown Precipitation)
- PY (Spray)
- SQ (Squall)
- DR (Low Drifting)
- SH (Shower)
- FZ (Freezing)
- MI (shallow)
- PR (Partial)
- BC (Patches)
- BL (Blowing)
- VC (Vicinity)
- DU (Widespread Dust)
- DS (Duststorm)
- PO (Sand Whirls)
- SA (Sand)
- SS (Sandstorm)
- ' ' (Moderate)

In [245]:
# first, create a function to extract hot and dry weather conditions, cold and wet weather conditions, and normal conditions:

cold_wet = ['FC','TS','GR','RA','DZ','SN','SG','GS','PL','IC','FG','BR','UP',
              'PY','SQ','DR','SH','FZ','MI','PR','BC','BL','VC']
# FC (Tornado), FC (Funnel Cloud), TS (Thunderstorm),
# GR (Hail), RA (Rain), DZ (Drizzle), SN (Snow), SG (Snow Grains), GS (Small Hail), PL (Ice Pellets), IC (Ice Crystals),
# FG (Fog), BR (Mist), UP (Unknown Precipitation), PY (Spray), SQ (Squall), DR (Low Drifting), SH (Shower), FZ (Freezing),
# MI (shallow), PR (Partial), BC (Patches), BL (Blowing), VC (Vicinity)

hot_dry = ['HZ','FU','VA','DU','DS','PO','SA','SS']
# HZ (Haze), FU (Smoke), VA (Volcanic Ash), DU (Widespread Dust), DS (Duststorm), PO (Sand Whirls), SA (Sand), SS (Sandstorm)

def weather_type(codesum):
    if len(codesum) > 2:
        codesum = codesum[:2]
    if codesum in cold_wet:
        return 'cold_wet'
    elif codesum == ' ':
        return 'normal'
    else:
        return 'hot_dry'

In [246]:
train_spray_weat['Weather'] = train_spray_weat['CodeSum'].apply(lambda x: weather_type(x))
train_spray_weat.head()

,Address,Date,Latitude,Longitude,NumMosquitos,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,most_recent_spray,WnvPresent,Station,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Month,Year,Weather
0,"7900 West Foster Avenue, Chicago, IL 60656, USA",2007-05-29,41.974,-87.825,1,0,1,0,0,0,0,3650,0,1,88,60,74,58,65,0,9,BR HZ,0.00,29.39,30.11,5.8,18,6.5,5,2007,cold_wet
1,"7900 West Foster Avenue, Chicago, IL 60656, USA",2007-05-29,41.974,-87.825,4,0,0,1,0,0,0,3650,0,1,88,60,74,58,65,0,9,BR HZ,0.00,29.39,30.11,5.8,18,6.5,5,2007,cold_wet
2,"7900 West Foster Avenue, Chicago, IL 60656, USA",2007-06-05,41.974,-87.825,1,0,1,0,0,0,0,3650,0,1,64,47,56,48,51,9,0,RA BR,0.42,29.10,29.79,5.2,5,7.6,6,2007,cold_wet
3,"7900 West Foster Avenue, Chicago, IL 60656, USA",2007-06-05,41.974,-87.825,2,0,0,1,0,0,0,3650,0,1,64,47,56,48,51,9,0,RA BR,0.42,29.10,29.79,5.2,5,7.6,6,2007,cold_wet
4,"7900 West Foster Avenue, Chicago, IL 60656, USA",2007-06-26,41.974,-87.825,1,0,0,1,0,0,0,3650,0,1,92,70,81,69,72,0,16,TSRA BR HZ VCTS,0.16,29.39,30.11,6.1,22,7.1,6,2007,cold_wet


In [247]:
train_spray_weat['Weather'].value_counts(normalize=True)

normal      0.571197
cold_wet    0.411384
hot_dry     0.017419
Name: Weather, dtype: float64

In [248]:
# Get dummies for weather type:

train_spray_weat = pd.get_dummies(train_spray_weat, columns=['Weather'], drop_first = True)
train_spray_weat.head().T

,0,1,2,3,4
Address,"7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA"
Date,2007-05-29 00:00:00,2007-05-29 00:00:00,2007-06-05 00:00:00,2007-06-05 00:00:00,2007-06-26 00:00:00
Latitude,41.974,41.974,41.974,41.974,41.974
Longitude,-87.825,-87.825,-87.825,-87.825,-87.825
NumMosquitos,1,4,1,2,1
Species_CULEX PIPIENS,0,0,0,0,0
Species_CULEX PIPIENS/RESTUANS,1,0,1,0,0
Species_CULEX RESTUANS,0,1,0,1,1
Species_CULEX SALINARIUS,0,0,0,0,0
Species_CULEX TARSALIS,0,0,0,0,0


In [249]:
# save DataFrame:

train_spray_weat.to_csv('output/train_spray_weat.csv',index=False)

In [250]:
print(train_spray_weat.shape)
print(train_spray_weat.columns)

(10506, 32)
Index(['Address', 'Date', 'Latitude', 'Longitude', 'NumMosquitos', 'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS', 'Species_CULEX RESTUANS', 'Species_CULEX SALINARIUS', 'Species_CULEX TARSALIS', 'Species_CULEX TERRITANS', 'most_recent_spray', 'WnvPresent', 'Station', 'Tmax', 'Tmin', 'Tavg', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'CodeSum', 'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed', 'Month', 'Year', 'Weather_hot_dry', 'Weather_normal'], dtype='object')


# Clean test dataset

In [251]:
test = pd.read_csv('input/test.csv')
test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [252]:
test.drop(columns = ['Address','Block','Street','AddressNumberAndStreet', 'Trap', 'AddressAccuracy'], inplace = True)
test = pd.get_dummies(test, columns=['Species'], drop_first=True)    
test['Date'] = pd.to_datetime(test['Date'])
test['Latitude'] = test['Latitude'].round(3)
test['Longitude'] = test['Longitude'].round(3)
test.head()

,Id,Date,Latitude,Longitude,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX
0,1,2008-06-11,41.955,-87.801,0,1,0,0,0,0,0
1,2,2008-06-11,41.955,-87.801,0,0,1,0,0,0,0
2,3,2008-06-11,41.955,-87.801,1,0,0,0,0,0,0
3,4,2008-06-11,41.955,-87.801,0,0,0,1,0,0,0
4,5,2008-06-11,41.955,-87.801,0,0,0,0,0,1,0


In [253]:
spray.head()

,Longitude,Latitude,count,min,max,diff
696,-87.811,41.997,3,2011-09-07,2013-09-05,729
1804,-87.784,41.957,3,2013-08-15,2013-08-16,1
1858,-87.783,41.965,3,2013-08-15,2013-08-16,1
2789,-87.750,41.911,2,2013-08-15,2013-08-16,1
1801,-87.784,41.954,4,2013-08-15,2013-08-16,1


In [254]:
print(test.shape)
print(spray.shape)

(116293, 11)
(6517, 6)


In [255]:
test1 = pd.merge(test, spray, on=['Latitude','Longitude'], how='right', indicator=False)
    
test1['Date'] = pd.to_datetime(test1['Date'])
test1['diffmin'] = test1['Date'] - test1['min']
test1['diffmin'] = test1['diffmin'].dt.days
test1['diffmax'] = test1['Date'] - test1['max']
test1['diffmax'] = test1['diffmax'].dt.days
test1['most_recent_spray'] = 3650

In [256]:
test1.head()

,Id,Date,Latitude,Longitude,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX,count,min,max,diff,diffmin,diffmax,most_recent_spray
0,17.0,2008-06-11,41.974,-87.825,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,2011-09-07,2011-09-07,0,-1183.0,-1183.0,3650
1,18.0,2008-06-11,41.974,-87.825,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,2011-09-07,2011-09-07,0,-1183.0,-1183.0,3650
2,19.0,2008-06-11,41.974,-87.825,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2011-09-07,2011-09-07,0,-1183.0,-1183.0,3650
3,20.0,2008-06-11,41.974,-87.825,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,2011-09-07,2011-09-07,0,-1183.0,-1183.0,3650
4,21.0,2008-06-11,41.974,-87.825,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,2011-09-07,2011-09-07,0,-1183.0,-1183.0,3650


In [257]:
test1.shape

(23288, 18)

In [258]:
# rename previous train_spray dataframe to prevent confusion:
test1_right = test1

# create inner merge dataframe
test1_inner = pd.merge(test, spray, on = ['Longitude','Latitude'], how='inner',indicator=False)

# perform the same transformations as above
test1_inner['Date'] = pd.to_datetime(test1_inner['Date'])
test1_inner['diffmin'] = test1_inner['Date'] - test1_inner['min']
test1_inner['diffmin'] = test1_inner['diffmin'].dt.days
test1_inner['diffmax'] = test1_inner['Date'] - test1_inner['max']
test1_inner['diffmax'] = test1_inner['diffmax'].dt.days

test1_inner.head()

,Id,Date,Latitude,Longitude,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX,count,min,max,diff,diffmin,diffmax
0,17,2008-06-11,41.974,-87.825,0,1,0,0,0,0,0,1,2011-09-07,2011-09-07,0,-1183,-1183
1,18,2008-06-11,41.974,-87.825,0,0,1,0,0,0,0,1,2011-09-07,2011-09-07,0,-1183,-1183
2,19,2008-06-11,41.974,-87.825,1,0,0,0,0,0,0,1,2011-09-07,2011-09-07,0,-1183,-1183
3,20,2008-06-11,41.974,-87.825,0,0,0,1,0,0,0,1,2011-09-07,2011-09-07,0,-1183,-1183
4,21,2008-06-11,41.974,-87.825,0,0,0,0,0,1,0,1,2011-09-07,2011-09-07,0,-1183,-1183


In [259]:
test1_inner.shape

(16793, 17)

In [260]:
test1_inner['most_recent_spray'] = test1_inner.apply(lambda x: recent_spray(x['diffmin'], x['diffmax']),axis=1)

In [261]:
test1_right[test1_right.isna().any(axis=1)]

# null entries start appearing from index 16793 onwards

,Id,Date,Latitude,Longitude,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX,count,min,max,diff,diffmin,diffmax,most_recent_spray
16793,NaN,NaT,41.997,-87.811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2011-09-07,2013-09-05,729,NaN,NaN,3650
16794,NaN,NaT,41.957,-87.784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2013-08-15,2013-08-16,1,NaN,NaN,3650
16795,NaN,NaT,41.965,-87.783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2013-08-15,2013-08-16,1,NaN,NaN,3650
16796,NaN,NaT,41.911,-87.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2013-08-15,2013-08-16,1,NaN,NaN,3650
16797,NaN,NaT,41.954,-87.784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2013-08-15,2013-08-16,1,NaN,NaN,3650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23283,NaN,NaT,42.008,-87.776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2013-07-17,2013-07-17,0,NaN,NaN,3650
23284,NaN,NaT,42.007,-87.776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2013-07-17,2013-07-17,0,NaN,NaN,3650
23285,NaN,NaT,42.006,-87.776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2013-07-17,2013-07-17,0,NaN,NaN,3650
23286,NaN,NaT,42.005,-87.776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2013-07-17,2013-07-17,0,NaN,NaN,3650


In [262]:
test1 = pd.concat([test1_inner, test1_right.loc[16793:,:]])
test1[test1.isna().any(axis=1)]

,Id,Date,Latitude,Longitude,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX,count,min,max,diff,diffmin,diffmax,most_recent_spray
16793,NaN,NaT,41.997,-87.811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2011-09-07,2013-09-05,729,NaN,NaN,3650
16794,NaN,NaT,41.957,-87.784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2013-08-15,2013-08-16,1,NaN,NaN,3650
16795,NaN,NaT,41.965,-87.783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2013-08-15,2013-08-16,1,NaN,NaN,3650
16796,NaN,NaT,41.911,-87.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2013-08-15,2013-08-16,1,NaN,NaN,3650
16797,NaN,NaT,41.954,-87.784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2013-08-15,2013-08-16,1,NaN,NaN,3650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23283,NaN,NaT,42.008,-87.776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2013-07-17,2013-07-17,0,NaN,NaN,3650
23284,NaN,NaT,42.007,-87.776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2013-07-17,2013-07-17,0,NaN,NaN,3650
23285,NaN,NaT,42.006,-87.776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2013-07-17,2013-07-17,0,NaN,NaN,3650
23286,NaN,NaT,42.005,-87.776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2013-07-17,2013-07-17,0,NaN,NaN,3650


In [263]:
print(test1.shape[0])
print(test1[test1.isna().any(axis=1)].shape[0])
print(test1[test1.isna().any(axis=1)].shape[0] / test1.shape[0])

23288
6495
0.2788990037787702


In [264]:
test1.isnull().sum()

Id                                6495
Date                              6495
Latitude                             0
Longitude                            0
Species_CULEX PIPIENS             6495
Species_CULEX PIPIENS/RESTUANS    6495
Species_CULEX RESTUANS            6495
Species_CULEX SALINARIUS          6495
Species_CULEX TARSALIS            6495
Species_CULEX TERRITANS           6495
Species_UNSPECIFIED CULEX         6495
count                                0
min                                  0
max                                  0
diff                                 0
diffmin                           6495
diffmax                           6495
most_recent_spray                    0
dtype: int64

In [265]:
test.shape

(116293, 11)

Disregarding `diffmin` and `diffmax` features which will be dropped anyway, there are 6495 null entries/rows after the merging and preprocessing. This accounts for close to 28% of the whole test dataset.

However, more importantly, cleaning and preprocessing the test dataset in the same manner as for the train dataset (right merge followed by inner merge) results in a dataset with only 23288 entries. Kaggle submission stipulates that 116293 prediction rows must be maintained (i.e. original number of entries preserved, no dropping at all).

Hence, we will be merging test dataset with spray dataset in a slightly different way.

In [266]:
test.head()

,Id,Date,Latitude,Longitude,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX
0,1,2008-06-11,41.955,-87.801,0,1,0,0,0,0,0
1,2,2008-06-11,41.955,-87.801,0,0,1,0,0,0,0
2,3,2008-06-11,41.955,-87.801,1,0,0,0,0,0,0
3,4,2008-06-11,41.955,-87.801,0,0,0,1,0,0,0
4,5,2008-06-11,41.955,-87.801,0,0,0,0,0,1,0


In [267]:
spray.head()

,Longitude,Latitude,count,min,max,diff
696,-87.811,41.997,3,2011-09-07,2013-09-05,729
1804,-87.784,41.957,3,2013-08-15,2013-08-16,1
1858,-87.783,41.965,3,2013-08-15,2013-08-16,1
2789,-87.750,41.911,2,2013-08-15,2013-08-16,1
1801,-87.784,41.954,4,2013-08-15,2013-08-16,1


In [268]:
# left merge instead, to preserve number of entries:

test1 = pd.merge(test, spray, on=['Latitude','Longitude'], how='left')
test1.shape

(116293, 15)

In [269]:
test1['Date'] = pd.to_datetime(test1['Date'])
test1['most_recent_spray'] = test1['Date'] - test1['min']
test1['most_recent_spray'] = test1['most_recent_spray'].dt.days

In [270]:
test1.head()

,Id,Date,Latitude,Longitude,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX,count,min,max,diff,most_recent_spray
0,1,2008-06-11,41.955,-87.801,0,1,0,0,0,0,0,NaN,NaT,NaT,NaN,NaN
1,2,2008-06-11,41.955,-87.801,0,0,1,0,0,0,0,NaN,NaT,NaT,NaN,NaN
2,3,2008-06-11,41.955,-87.801,1,0,0,0,0,0,0,NaN,NaT,NaT,NaN,NaN
3,4,2008-06-11,41.955,-87.801,0,0,0,1,0,0,0,NaN,NaT,NaT,NaN,NaN
4,5,2008-06-11,41.955,-87.801,0,0,0,0,0,1,0,NaN,NaT,NaT,NaN,NaN


In [271]:
# instead of applying the 'recent_spray(diffmin, diffmax)' function to populate 'most_recent_spray' feature,
# fill up the column by

test1['most_recent_spray'] = test1['most_recent_spray'].apply(lambda x: x if x > 0 else 3650)

**Intuition/assumption:**

- For entries with no spray value (null values), and entries with negative recent spray days (which means the spray only occurred after the data collection date), we impute the most recent spray to be 3650 days, or 10 years ago.

In [272]:
test1[test1['most_recent_spray']<3650]['most_recent_spray'].describe()

count    3887.000000
mean      458.546951
std       257.046241
min       273.000000
25%       329.000000
50%       359.000000
75%       399.000000
max      1121.000000
Name: most_recent_spray, dtype: float64

In [273]:
test1.most_recent_spray.value_counts().sort_values(ascending=False)

3650.0    112406
350.0        129
322.0        128
343.0        128
315.0        128
357.0        128
329.0        128
336.0        128
371.0        121
378.0        120
392.0        113
308.0        104
399.0        104
364.0        104
301.0        104
413.0         88
294.0         88
385.0         80
406.0         72
386.0         64
372.0         56
379.0         56
358.0         48
337.0         48
344.0         48
351.0         48
407.0         40
420.0         40
287.0         40
365.0         40
1065.0        36
1072.0        35
1058.0        33
324.0         33
1023.0        33
331.0         33
1079.0        33
288.0         32
1051.0        32
345.0         32
1030.0        32
296.0         32
1094.0        32
338.0         32
282.0         32
1107.0        32
1086.0        32
1114.0        32
1121.0        32
352.0         32
275.0         32
359.0         32
1100.0        32
306.0         32
316.0         32
1044.0        32
317.0         32
1002.0        32
310.0         

In [274]:
print(test1.loc[test1['most_recent_spray'] < 3650].shape[0])
print(test1.shape[0])

3887
116293


In [275]:
print(test1.loc[test1['most_recent_spray'] < 3650].shape[0] / test.shape[0])

0.033424195781345395


There isn't too much variation in the `most_recent_spray` data (only 3.34% of the values are NOT 3650), but we will just make use of it.

In [276]:
test1.shape

(116293, 16)

In [277]:
test1.isnull().sum()

Id                                    0
Date                                  0
Latitude                              0
Longitude                             0
Species_CULEX PIPIENS                 0
Species_CULEX PIPIENS/RESTUANS        0
Species_CULEX RESTUANS                0
Species_CULEX SALINARIUS              0
Species_CULEX TARSALIS                0
Species_CULEX TERRITANS               0
Species_UNSPECIFIED CULEX             0
count                             99500
min                               99500
max                               99500
diff                              99500
most_recent_spray                     0
dtype: int64

In [278]:
# proceed to the next phase of merging with weather dataset:

test1.drop(columns = ['min', 'max', 'count', 'diff'], inplace=True)
test1.head().T

,0,1,2,3,4
Id,1,2,3,4,5
Date,2008-06-11 00:00:00,2008-06-11 00:00:00,2008-06-11 00:00:00,2008-06-11 00:00:00,2008-06-11 00:00:00
Latitude,41.955,41.955,41.955,41.955,41.955
Longitude,-87.801,-87.801,-87.801,-87.801,-87.801
Species_CULEX PIPIENS,0,0,1,0,0
Species_CULEX PIPIENS/RESTUANS,1,0,0,0,0
Species_CULEX RESTUANS,0,1,0,0,0
Species_CULEX SALINARIUS,0,0,0,1,0
Species_CULEX TARSALIS,0,0,0,0,0
Species_CULEX TERRITANS,0,0,0,0,1


In [279]:
test1.dtypes

Id                                         int64
Date                              datetime64[ns]
Latitude                                 float64
Longitude                                float64
Species_CULEX PIPIENS                      uint8
Species_CULEX PIPIENS/RESTUANS             uint8
Species_CULEX RESTUANS                     uint8
Species_CULEX SALINARIUS                   uint8
Species_CULEX TARSALIS                     uint8
Species_CULEX TERRITANS                    uint8
Species_UNSPECIFIED CULEX                  uint8
most_recent_spray                        float64
dtype: object

In [280]:
weather.dtypes

Station                 int64
Date           datetime64[ns]
Tmax                    int64
Tmin                    int64
Tavg                   object
Depart                 object
DewPoint                int64
WetBulb                object
Heat                   object
Cool                   object
Sunrise                object
Sunset                 object
CodeSum                object
Depth                  object
Water1                 object
SnowFall               object
PrecipTotal            object
StnPressure            object
SeaLevel               object
ResultSpeed           float64
ResultDir               int64
AvgSpeed               object
dtype: object

In [281]:
weather['Date'] = pd.to_datetime(weather['Date'])

In [282]:
test1['Station'] = test1.apply(lambda x: nearest_stat(x['Latitude'], x['Longitude']),axis=1)

test1 = pd.merge(test1, weather, left_on=['Date','Station'], right_on=['Date','Station'], how='left')

test1.head()

,Id,Date,Latitude,Longitude,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX,most_recent_spray,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2008-06-11,41.955,-87.801,0,1,0,0,0,0,0,3650.0,1,86,61,74,7,56,64,0,9,0416,1926,,0,M,0.0,0.00,29.28,29.99,8.9,18,10.0
1,2,2008-06-11,41.955,-87.801,0,0,1,0,0,0,0,3650.0,1,86,61,74,7,56,64,0,9,0416,1926,,0,M,0.0,0.00,29.28,29.99,8.9,18,10.0
2,3,2008-06-11,41.955,-87.801,1,0,0,0,0,0,0,3650.0,1,86,61,74,7,56,64,0,9,0416,1926,,0,M,0.0,0.00,29.28,29.99,8.9,18,10.0
3,4,2008-06-11,41.955,-87.801,0,0,0,1,0,0,0,3650.0,1,86,61,74,7,56,64,0,9,0416,1926,,0,M,0.0,0.00,29.28,29.99,8.9,18,10.0
4,5,2008-06-11,41.955,-87.801,0,0,0,0,0,1,0,3650.0,1,86,61,74,7,56,64,0,9,0416,1926,,0,M,0.0,0.00,29.28,29.99,8.9,18,10.0


In [283]:
test1.drop(columns = ['Sunrise', 'Sunset', 'Water1', 'Depart', 'Depth','SnowFall'], inplace=True)
test1.shape

(116293, 27)

In [284]:
test1['Tavg'] = test1['Tavg'].astype(int)
test1['Heat'] = test1['Heat'].astype(int)
test1['Cool'] = test1['Cool'].astype(int)
test1['SeaLevel'] = test1['SeaLevel'].astype(float)
test1['AvgSpeed'] = test1['AvgSpeed'].astype(float)
test1['WetBulb'] = test1.apply(lambda x: fill_wet_bulb(x['Tavg'], x['DewPoint'], x['WetBulb']), axis=1)
test1['WetBulb'] = test1['WetBulb'].astype(int)
test1['PrecipTotal'] = test1['PrecipTotal'].apply(lambda x: 0.002 if x=='  T' else x)
mode = st.mode(test1['PrecipTotal'])
test1['PrecipTotal'] = test1['PrecipTotal'].apply(lambda x: mode if x=='M' else x)
test1['PrecipTotal'] = test1['PrecipTotal'].astype(float)
mode = st.mode(test1['StnPressure'])
test1['StnPressure'] = test1['StnPressure'].apply(lambda x: mode if x=='M' else x)
test1['StnPressure'] = test1['StnPressure'].astype(float)

test1.head()

,Id,Date,Latitude,Longitude,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX,most_recent_spray,Station,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2008-06-11,41.955,-87.801,0,1,0,0,0,0,0,3650.0,1,86,61,74,56,64,0,9,,0.0,29.28,29.99,8.9,18,10.0
1,2,2008-06-11,41.955,-87.801,0,0,1,0,0,0,0,3650.0,1,86,61,74,56,64,0,9,,0.0,29.28,29.99,8.9,18,10.0
2,3,2008-06-11,41.955,-87.801,1,0,0,0,0,0,0,3650.0,1,86,61,74,56,64,0,9,,0.0,29.28,29.99,8.9,18,10.0
3,4,2008-06-11,41.955,-87.801,0,0,0,1,0,0,0,3650.0,1,86,61,74,56,64,0,9,,0.0,29.28,29.99,8.9,18,10.0
4,5,2008-06-11,41.955,-87.801,0,0,0,0,0,1,0,3650.0,1,86,61,74,56,64,0,9,,0.0,29.28,29.99,8.9,18,10.0


In [285]:
test1.dtypes

Id                                         int64
Date                              datetime64[ns]
Latitude                                 float64
Longitude                                float64
Species_CULEX PIPIENS                      uint8
Species_CULEX PIPIENS/RESTUANS             uint8
Species_CULEX RESTUANS                     uint8
Species_CULEX SALINARIUS                   uint8
Species_CULEX TARSALIS                     uint8
Species_CULEX TERRITANS                    uint8
Species_UNSPECIFIED CULEX                  uint8
most_recent_spray                        float64
Station                                    int64
Tmax                                       int64
Tmin                                       int64
Tavg                                       int64
DewPoint                                   int64
WetBulb                                    int64
Heat                                       int64
Cool                                       int64
CodeSum             

In [286]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [288]:
test1['Month'] = test1['Date'].apply(lambda x: x.to_pydatetime().month)
test1['Year'] = test1['Date'].apply(lambda x: x.to_pydatetime().year)

test1['Weather'] = test1['CodeSum'].apply(lambda x: weather_type(x))
test1.head()

,Id,Date,Latitude,Longitude,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX,most_recent_spray,Station,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Month,Year,Weather
0,1,2008-06-11,41.955,-87.801,0,1,0,0,0,0,0,3650.0,1,86,61,74,56,64,0,9,,0.0,29.28,29.99,8.9,18,10.0,6,2008,normal
1,2,2008-06-11,41.955,-87.801,0,0,1,0,0,0,0,3650.0,1,86,61,74,56,64,0,9,,0.0,29.28,29.99,8.9,18,10.0,6,2008,normal
2,3,2008-06-11,41.955,-87.801,1,0,0,0,0,0,0,3650.0,1,86,61,74,56,64,0,9,,0.0,29.28,29.99,8.9,18,10.0,6,2008,normal
3,4,2008-06-11,41.955,-87.801,0,0,0,1,0,0,0,3650.0,1,86,61,74,56,64,0,9,,0.0,29.28,29.99,8.9,18,10.0,6,2008,normal
4,5,2008-06-11,41.955,-87.801,0,0,0,0,0,1,0,3650.0,1,86,61,74,56,64,0,9,,0.0,29.28,29.99,8.9,18,10.0,6,2008,normal


In [289]:
test1 = pd.get_dummies(test1, columns=['Weather'], drop_first = True)

In [290]:
test1.to_csv('output/test.csv',index=False)